In [9]:
#/usr/bin/env python
#-*- coding:utf-8-*-
import requests
import json
import os
import urllib
import os.path
import pickle
import pandas as pd
class MovieDB(object):
    def __init__(self,url = 'http://api.themoviedb.org/3',
                    headers = {'Accept':'application/json'},
                    payload = {'api_key':'e0ec617d1ba0c2c3adb4f43df47f9ab4'}):
        self.url = url
        self.headers = headers
        self.payload = payload
        
    def getImageBasePath(self,size='w185'):
        proxies = { "http": "http://localhost:1080", "https": "http://1ocalhost:1080", }   
        response = requests.get(self.url+'/configuration',params=self.payload,headers=self.headers)
        response = json.loads(response.text)
        base_url = response['images']['base_url']+'w185'
        return base_url
    
    def get_poster(self,imdb):
        #query themovie.org API for movie poster path.
        base_url = self.getImageBasePath()
        file_path = ''
        imdb_id = 'tt0{0}'.format(imdb)
        movie_url = self.url+'/movie/{:}/images'.format(imdb_id)
        response = requests.get(movie_url,params=self.payload,headers=self.headers)
        try:
            file_path = json.loads(response.text)['posters'][0]['file_path']
        except:
            print('Failed to get url for imdb: {0}'.format(imdb))
        return base_url+file_path
    
    def getIDs(self,link_f="./ml-latest-small/links.csv",download_posters=1000):
        df_id = pd.read_csv(link_f,sep=',')
        idx_to_mv = {}
        for row in df_id.itertuples():
            idx_to_mv[row[1]-1] = row[2]
        mvs = [0]*len(idx_to_mv.keys())
        for i in range(len(mvs)):
            if i in idx_to_mv.keys() and len(str(idx_to_mv[i])) == 6:
                mvs[i] = idx_to_mv[i]
        mvs = list(filter(lambda imdb:imdb!=0,mvs))
#         shuffle_index = range(len(mvs))
#         np.random.shuffle(shuffle_index)
#         mvs = [mvs[i] for i in mvs]
        mvs = mvs[:download_posters]
        return mvs
    
    def down_poster(self,mvs,poster_pt='./poster/'):
        base_url = self.getImageBasePath()
        total_mvs = len(mvs)
        URL = [0]*total_mvs
        URL_IMDB = {'url':[],'imdb':[],'pic':[]}
        i=0
        for m in mvs:
            if(os.path.exists(poster_pt+str(i)+'.jpg')):
                print('Skip downloading exists jpg: {0}.jpg'.format(poster_pt+str(i)))
                i += 1
                continue
            URL[i] = self.get_poster(m)
            if(URL[i] == base_url):
                print('Bad imdb id: {0}'.format(m))
                mvs.remove(m)
                continue
            print('No.{0}: Downloading jpg(imdb {1}) {2}'.format(i,m,URL[i]))
            urllib.urlretrieve(URL[i],poster_pt+str(i)+'.jpg')
            URL_IMDB['url'].append(URL[i])
            URL_IMDB['imdb'].append(m)
            URL_IMDB['pic'].append(i)
            i += 1
        output = open('URL_IMDB.pkl', 'wb')
        # Pickle dictionary using protocol 0.
        pickle.dump(URL_IMDB, output)
        # Pickle the list using the highest protocol available.
        output.close()
        return URL_IMDB
if __name__ == '__main__':        
    movieDb = MovieDB()
    mvs = movieDb.getIDs()
    URL_IMDB = movieDb.down_poster(mvs)
#   url=movieDb.get_poster(mvs[0])

Skip downloading exists jpg: ./poster/0.jpg
Skip downloading exists jpg: ./poster/1.jpg
Skip downloading exists jpg: ./poster/2.jpg
Skip downloading exists jpg: ./poster/3.jpg
Skip downloading exists jpg: ./poster/4.jpg
Skip downloading exists jpg: ./poster/5.jpg
Skip downloading exists jpg: ./poster/6.jpg
Skip downloading exists jpg: ./poster/7.jpg
Skip downloading exists jpg: ./poster/8.jpg
Skip downloading exists jpg: ./poster/9.jpg
Skip downloading exists jpg: ./poster/10.jpg
Skip downloading exists jpg: ./poster/11.jpg
Skip downloading exists jpg: ./poster/12.jpg
Skip downloading exists jpg: ./poster/13.jpg
Skip downloading exists jpg: ./poster/14.jpg
Skip downloading exists jpg: ./poster/15.jpg
Skip downloading exists jpg: ./poster/16.jpg
Skip downloading exists jpg: ./poster/17.jpg
Skip downloading exists jpg: ./poster/18.jpg
Skip downloading exists jpg: ./poster/19.jpg
Skip downloading exists jpg: ./poster/20.jpg
Skip downloading exists jpg: ./poster/21.jpg
Skip downloading exi

Skip downloading exists jpg: ./poster/480.jpg
Skip downloading exists jpg: ./poster/481.jpg
Skip downloading exists jpg: ./poster/482.jpg
Skip downloading exists jpg: ./poster/483.jpg
Skip downloading exists jpg: ./poster/484.jpg
Skip downloading exists jpg: ./poster/485.jpg
Skip downloading exists jpg: ./poster/486.jpg
Skip downloading exists jpg: ./poster/487.jpg
Skip downloading exists jpg: ./poster/488.jpg
Skip downloading exists jpg: ./poster/489.jpg
Skip downloading exists jpg: ./poster/490.jpg
Skip downloading exists jpg: ./poster/491.jpg
Skip downloading exists jpg: ./poster/492.jpg
Skip downloading exists jpg: ./poster/493.jpg
Skip downloading exists jpg: ./poster/494.jpg
Skip downloading exists jpg: ./poster/495.jpg
Skip downloading exists jpg: ./poster/496.jpg
Skip downloading exists jpg: ./poster/497.jpg
Skip downloading exists jpg: ./poster/498.jpg
Skip downloading exists jpg: ./poster/499.jpg
Skip downloading exists jpg: ./poster/500.jpg
Skip downloading exists jpg: ./pos

In [11]:
from keras.preprocessing import image as kimage
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
import os
import numpy as np
from re import match
class Vgg16(object):
    def __init__(self,imageDir='./poster/',prediction_result="prediction_result.csv"):
        self.imageDir = imageDir
        self.prediction_result = prediction_result
        
    def save_predict_res(self,matrix_res):
        prediction_result=self.prediction_result
        pe = dict([(x,[]) for x in range(matrix_res.shape[0])])
        for i in range(matrix_res.shape[0]):
            pe[i].extend(matrix_res[i])
        df = pd.DataFrame(data=pe)
#         df.to_csv(prediction_result)

    def load_result_matrix(self):
        file = self.prediction_result
        if not os.path.exists(file):
            return None
        m_r = pd.read_csv(file,sep=',')
        f_r = np.zeros((m_r.shape[1]-1,m_r.shape[0]))
        for i in range(m_r.values.shape[1]-1):
            f_r[i] = m_r[str(i)].values.tolist()
        return f_r 

    def train(self):
        poster_pt = self.imageDir
        total_mvs = len([x for x in os.listdir(poster_pt) if match("\d*.jpg",x)])
#         total_mvs = 3000
        image = [0]*total_mvs
        x     = [0]*total_mvs
        for i in range(total_mvs):
            image[i] = kimage.load_img(poster_pt+str(i)+'.jpg',target_size=(224,224))
            x[i] = kimage.img_to_array(image[i])
            x[i] = np.expand_dims(x[i],axis=0)
            x[i] = preprocess_input(x[i])
        model = VGG16(include_top=False,weights='imagenet')
        prediction = [0]*total_mvs
        matrix_res = np.zeros([total_mvs,25088])
        for i in range(total_mvs):
            prediction[i]=model.predict(x[i]).ravel()
            matrix_res[i,:]=prediction[i]
    #     print model.predict(x[0])
        return matrix_res
    
    def simaility(self,matrix_res):
        similarity_deep = matrix_res.dot(matrix_res.T)
        norms = np.array([np.sqrt(np.diagonal(similarity_deep))])
#         print (norms.dot(norms.T)).shape
        similarity_deep = (similarity_deep/(norms*norms.T))
        return similarity_deep
    
    def topN(self,target_item,similarity_deep,N=5):
        similarity_deep = self.simaility(matrix_res)
        topN_list = [x for x in np.argsort(np.nan_to_num(similarity_deep)[target_item])[:-N-1:-1]]
        return topN_list
        
    def predictAll(self,similarity_deep):
        poster_pt = self.imageDir
        total_mvs = len([x for x in os.listdir(poster_pt) if match("\d*.jpg",x)])
#         total_mvs = 3000
        prediction_list = dict([(x,[]) for x in range(total_mvs)])
        for i in prediction_list.keys():
            prediction_list[i] = self.topN(i,similarity_deep)
        return prediction_list
    
if __name__ == '__main__':
    vgg16 = Vgg16()
    matrix_res = vgg16.load_result_matrix()
    if matrix_res is None:
        print "trian model....."
        matrix_res = vgg16.train()
        vgg16.save_predict_res(matrix_res)
        print "Saved prediction_res in %s" %vgg16.prediction_result
    similarity_deep = vgg16.simaility(matrix_res)
    np.save('similarity',similarity_deep)
#     predict_result = vgg16.predictAll(similarity_deep)

    
    
    
   

trian model.....
Saved prediction_res in prediction_result.csv


In [17]:
from IPython.display import Image
from IPython.display import display
from IPython.display import HTML
poster_pt = './poster/'
for j in range(10):
    images = ''
    mv = vgg16.topN(j,similarity_deep)
    for i in range(len(mv)):
        images+=("<img style='width: 100px; margin: 0px; float: left; border: 1px solid black;' src={0}.jpg />"
                .format(poster_pt+str(mv[i])))
    display(HTML(images))

In [113]:
f = open('test.txt','w+')
print >>f, "hello word"
f.close()

In [115]:
#序列化保存 对象
import pickle
import pprint, pickle
import pickle
output = open('predict.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(predict_result, output)
# Pickle the list using the highest protocol available.
output.close()

In [117]:
#序列化对象 恢复
import pprint, pickle
pkl_file = open('predict.pkl', 'rb')
data = pickle.load(pkl_file)
# pprint.pprint(data)
pkl_file.close()
type(data)

dict

In [122]:
type(None)

NoneType